In [1]:
from sklearn.model_selection import train_test_split


Выделение dev-корпуса, как в бейзлайне:

In [2]:
texts, ids = [], []
with open('train_reviews.txt', encoding = 'utf-8') as f:
    for line in f:
        text_id, text = line.rstrip('\r\n').split('\t')
        texts.append(text)
        ids.append(text_id)

In [3]:
train_texts, dev_texts, train_ids, dev_ids = train_test_split(texts, ids)


In [4]:
train_aspects, dev_aspects = [], []
with open('train_aspects.txt', encoding = 'utf-8') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_aspects.append(line)
        if text_id in dev_ids:
            dev_aspects.append(line)

In [5]:
train_sentiment, dev_sentiment = [], []
with open('train_cats.txt') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_sentiment.append(line)
        if text_id in dev_ids:
            dev_sentiment.append(line)

In [6]:
with open('train_split_aspects.txt', 'w') as f:
    for l in train_aspects:
        print(l, file=f)
with open('dev_aspects.txt', 'w') as f:
    for l in dev_aspects:
        print(l, file=f)
with open('train_split_reviews.txt', 'w') as f:
    for i, l in zip(train_ids, train_texts):
        print(i, l, sep="\t", file=f)
with open('dev_reviews.txt', 'w') as f:
    for i, l in zip(dev_ids, dev_texts):
        print(i, l, sep="\t", file=f)
with open('train_split_cats.txt', 'w') as f:
    for l in train_sentiment:
        print(l, file=f)
with open('dev_cats.txt', 'w') as f:
    for l in dev_sentiment:
        print(l, file=f)

In [7]:
import pandas as pd


In [8]:
import csv

In [12]:
train_asp = pd.read_csv(
    'train_split_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)
train_texts = pd.read_csv('train_split_reviews.txt', delimiter='\t', names=['text_id','text'])

In [13]:
train_asp

,text_id,category,mention,start,end,sentiment
0,30808,Whole,ресторане,16,25,neutral
1,30808,Interior,первом этаже,43,55,neutral
2,30808,Whole,руководству ресторана,124,145,positive
3,30808,Service,обслуживающему персоналу,147,171,positive
4,30808,Service,сотрудникам,189,200,positive
...,...,...,...,...,...,...
3561,16630,Service,обслуживание,85,97,positive
3562,16630,Food,Еда,99,102,positive
3563,16630,Service,персоналу,244,253,positive
3564,16630,Whole,ресторан,294,302,positive


In [14]:
train_texts

,text_id,text
0,2468,"Итак, начну с конца. ПОНРАВИЛОСЬ ВСЁ!!! Наверн..."
1,22975,"Недавно посетила данный ресторан,я согласна со..."
2,33912,Добрый день! Были с подружкой еще две недели н...
3,38772,Наша семья в восторге от данного ресторана!!! ...
4,9870,Семейное торжество отмечали в Амадеусе 30 мая ...
...,...,...
208,33816,Праздновали здесь юбилей подруги. Довольны оче...
209,18372,Ресторанчик очень милый.бывала там не раз!Гото...
210,16676,Зашел в это местечко после работы поужинать. В...
211,36926,Добрый день! Первый раз пишу отзыв о ресторане...


In [15]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [16]:
punct = '/.,:;!?"<>(){}[]*-+'
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        if t not in punct:
            lemmas.append(
                m.parse(t)[0].normal_form
            )
    return ' '.join(lemmas)

Добавление столбца с текстами отзывов, токенизированными и лемматизированными

In [17]:
train_texts['texts_normalized'] = [normalize_text(t) for t in train_texts['text']]
train_texts

,text_id,text,texts_normalized
0,2468,"Итак, начну с конца. ПОНРАВИЛОСЬ ВСЁ!!! Наверн...",итак начать с конец понравиться всё наверное я...
1,22975,"Недавно посетила данный ресторан,я согласна со...",недавно посетить данный ресторан я согласный с...
2,33912,Добрый день! Были с подружкой еще две недели н...,добрый день быть с подружка ещё два неделя наз...
3,38772,Наша семья в восторге от данного ресторана!!! ...,наш семья в восторг от данный ресторан жить не...
4,9870,Семейное торжество отмечали в Амадеусе 30 мая ...,семейный торжество отмечать в амадеуса 30 май ...
...,...,...,...
208,33816,Праздновали здесь юбилей подруги. Довольны оче...,праздновать здесь юбилей подруга довольный оче...
209,18372,Ресторанчик очень милый.бывала там не раз!Гото...,ресторанчик очень милый бывать там не раз гото...
210,16676,Зашел в это местечко после работы поужинать. В...,зайти в это местечко после работа поужинать вс...
211,36926,Добрый день! Первый раз пишу отзыв о ресторане...,добрый день первый раз писать отзыв о ресторан...


In [18]:
train_texts_list = train_texts['texts_normalized'].to_list()
train_texts_list_of_lists = []
for elem in train_texts_list:
    train_texts_list_of_lists.append(elem.split(' '))

In [19]:
train_asp['mention_lemmatized'] = [normalize_text(m) for m in train_asp['mention']]
train_asp

,text_id,category,mention,start,end,sentiment,mention_lemmatized
0,30808,Whole,ресторане,16,25,neutral,ресторан
1,30808,Interior,первом этаже,43,55,neutral,первый этаж
2,30808,Whole,руководству ресторана,124,145,positive,руководство ресторан
3,30808,Service,обслуживающему персоналу,147,171,positive,обслуживающий персонал
4,30808,Service,сотрудникам,189,200,positive,сотрудник
...,...,...,...,...,...,...,...
3561,16630,Service,обслуживание,85,97,positive,обслуживание
3562,16630,Food,Еда,99,102,positive,еда
3563,16630,Service,персоналу,244,253,positive,персонал
3564,16630,Whole,ресторан,294,302,positive,ресторан


In [20]:
from collections import defaultdict, Counter


In [21]:
def get_mention_category(data, cat_type):
    mention_categories = data.value_counts(subset=['mention_lemmatized', cat_type])
    mention_categories_dict = defaultdict(dict)
    for key, value in mention_categories.items():
        mention_categories_dict[key[0]][key[1]] = value
    return {k: Counter(v).most_common(1)[0][0] for k, v in mention_categories_dict.items()}

Как в бейзлайне, подсчет самой частотной категории для каждого индикатора аспекта:

In [22]:
counted_cat_for_mention = get_mention_category(train_asp, 'category')
counted_cat_for_mention

{'ресторан': 'Whole',
 'обслуживание': 'Service',
 'интерьер': 'Interior',
 'официант': 'Service',
 'заведение': 'Whole',
 'кухня': 'Food',
 'место': 'Whole',
 'блюдо': 'Food',
 'еда': 'Food',
 'цена': 'Price',
 'персонал': 'Service',
 'официантка': 'Service',
 'порция': 'Food',
 'пиво': 'Food',
 'салат': 'Food',
 'меню': 'Food',
 'атмосфера': 'Interior',
 'десерт': 'Food',
 'встретить': 'Service',
 'девушка': 'Service',
 'музыка': 'Interior',
 'обстановка': 'Interior',
 'администратор': 'Service',
 'обслуживать': 'Service',
 'сервис': 'Service',
 'зал': 'Interior',
 'горячий': 'Food',
 'ждать': 'Service',
 'кафе': 'Whole',
 'паста': 'Food',
 'напиток': 'Food',
 'мясо': 'Food',
 'вино': 'Food',
 'вечер': 'Whole',
 'принести': 'Service',
 'столик': 'Interior',
 'поесть': 'Food',
 'соус': 'Food',
 'пицца': 'Food',
 'суп': 'Food',
 'живой музыка': 'Interior',
 'ролл': 'Food',
 'заказать': 'Service',
 'молодой человек': 'Service',
 'закуска': 'Food',
 'приготовить': 'Food',
 'заказ': 'Serv

In [23]:
import math


In [24]:
def tfidf(corpus):
    words_list = []
    for doc in corpus:
        for word in doc:
            word_counter = 0
            for doc1 in corpus:
                if word in doc1:
                    word_counter +=1 
            tf = doc.count(word)/len(doc)
            idf = math.log(len(corpus)/word_counter)
            tf_idf = tf*idf
            word_tuple = [word, tf_idf]
            words_list.append(word_tuple)
    return words_list

In [25]:
tfidf_data = tfidf(train_texts_list_of_lists)
tfidf_data[0:100]

[['итак', 0.0247491450482182],
 ['начать', 0.02292802600270439],
 ['с', 0.0018503383464409026],
 ['конец', 0.01875872060485249],
 ['понравиться', 0.013973767269876751],
 ['всё', 0.0067703512708671325],
 ['наверное', 0.02084337330377844],
 ['я', 0.008682872911763319],
 ['консерватор', 0.038849943229778444],
 ['так', 0.017099591026218313],
 ['к', 0.005850835319629596],
 ['любить', 0.019726339391986716],
 ['заведение', 0.007096127036489446],
 ['именно', 0.03663882350222042],
 ['такой', 0.008713109292389517],
 ['план', 0.03382713757354696],
 ['хороший', 0.004403637497123627],
 ['интерьер', 0.0044663770822186595],
 ['негромкий', 0.02586617896000993],
 ['музыка', 0.010797761991315465],
 ['спокойный', 0.017513428887992642],
 ['атмосфера', 0.013296606094878722],
 ['вежливый', 0.01396597797988608],
 ['и', 0.0012049174062083291],
 ['учтивый', 0.030888984616241417],
 ['персонал', 0.009552470274455617],
 ['решить', 0.011940000717428387],
 ['отметить', 0.014203585391646883],
 ['свой', 0.00906100218

Беру 1000 вхождений с самым большим tf * idf (частотны в пределах документа, в корпусе - не очень). Можно видеть, что это в основном специальная лексика

In [28]:
tf_idf_df = pd.DataFrame(tfidf_data, columns = ['word', 'word_tfidf'])
tf_idf_df.sort_values(by="word_tfidf", ascending=False).head(1000)

,word,word_tfidf
20693,палочка,0.176156
20742,палочка,0.176156
7557,майкл,0.147288
7636,майкл,0.147288
7593,майкл,0.147288
...,...,...
19801,важный,0.055359
19931,важный,0.055359
6254,ужинать,0.055359
6292,подавать,0.055359


In [27]:
tf_idf_df1 = tf_idf_df.sort_values(by="word_tfidf", ascending=False).head(2000)
rare_words = tf_idf_df1['word'].to_list()
rare_words

['палочка',
 'палочка',
 'майкл',
 'майкл',
 'майкл',
 'майкл',
 'майкл',
 'квас',
 'квас',
 'квас',
 'квас',
 'roll',
 'roll',
 'roll',
 'roll',
 '260р',
 '260р',
 'молодец',
 'молодец',
 'случайный',
 'случайный',
 'нельсон',
 'нельсон',
 'нельсон',
 'чукк',
 'чукк',
 'базилик',
 'базилик',
 'прежний',
 'прежний',
 'арагви',
 'арагви',
 'арагви',
 'выпечка',
 'выпечка',
 '–',
 '–',
 'харбин',
 'приморский',
 'харбин',
 'харбин',
 'приморский',
 'харбин',
 'приморский',
 'приморский',
 'долгожданный',
 'знающий',
 'привыкнуть',
 'осьминог',
 'осьминог',
 'осьминог',
 'крапива',
 'крапива',
 'салями',
 'салями',
 'салями',
 'пиво',
 'пиво',
 'пиво',
 'пиво',
 'пиво',
 'работа',
 'работа',
 'работа',
 'детский',
 'детский',
 'детский',
 'детский',
 'детский',
 'детский',
 'более',
 'более',
 'более',
 'банкет',
 'банкет',
 'банкет',
 'солёный',
 'солёный',
 'испортиться',
 'испортиться',
 'попадать',
 'расслаблять',
 'словно',
 'обед',
 'звучать',
 'запоминться',
 'упаковывать',
 'китай

In [42]:
with open('tfidf_data.txt', 'w') as f:
    f.write (' '.join(rare_words))

Скачиваем список мебели (поможет определять индикаторы аспекта Interior:

In [29]:
with open('furnit_list.txt', encoding = 'utf-8') as file:
    f = file.readlines()
    furniture_list = []
    for line in f:
        line = line.rstrip('\r\n').lower()
        furniture_list.append(line)
furniture_list[0:10]#список мебели, может пригодиться для аспекта Interior

['банкетка',
 'бар',
 'бержер',
 'библиотека',
 'бинбэг',
 'буфет',
 'буфетница',
 'бюро',
 'верстак',
 'вешалка']

Список блюд, пригодится при определении аспекта Food

In [30]:
with open('dishes_list_from_wiki.txt', encoding = 'utf-8') as file2:
    f2 = file2.readlines()
    dishes_list = []
    for line in f2:
        line = line.rstrip('\r\n').lower()
        dishes_list.append(line)
dishes_list[0:10] #список блюд (с Википедии)


['абнабат',
 'абон',
 'авголемоно',
 'агуапанела',
 'аджапсандали',
 'аджика',
 'адобо',
 'азу',
 'айвар',
 'айвовый мармелад']

Как в бейзлайне, аспектные слова с частотными тональными ярлыками 

In [31]:
counted_sent_for_mention = get_mention_category(train_asp, 'sentiment')
counted_sent_for_mention

{'интерьер': 'positive',
 'ресторан': 'positive',
 'обслуживание': 'positive',
 'место': 'positive',
 'кухня': 'positive',
 'официант': 'positive',
 'заведение': 'positive',
 'персонал': 'positive',
 'блюдо': 'positive',
 'еда': 'positive',
 'цена': 'positive',
 'порция': 'positive',
 'атмосфера': 'positive',
 'обстановка': 'positive',
 'музыка': 'positive',
 'меню': 'positive',
 'встретить': 'positive',
 'официантка': 'positive',
 'девушка': 'positive',
 'обслуживать': 'positive',
 'администратор': 'positive',
 'пиво': 'positive',
 'салат': 'positive',
 'зал': 'positive',
 'десерт': 'positive',
 'напиток': 'neutral',
 'ждать': 'negative',
 'кафе': 'positive',
 'вечер': 'positive',
 'поесть': 'positive',
 'сервис': 'positive',
 'горячий': 'neutral',
 'живой музыка': 'positive',
 'молодой человек': 'positive',
 'приготовить': 'positive',
 'стол': 'positive',
 'столик': 'positive',
 'паста': 'positive',
 'провести время': 'positive',
 'заказ': 'positive',
 'скидка': 'positive',
 'помочь'

Также для определения тональных слов использую список слов, размеченных с точки зрения тональности:

In [32]:
sentiment_vocab = pd.read_csv('kartaslovsent.csv', sep = ';')
sentiment_vocab

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19
...,...,...,...,...,...,...,...,...
46122,ёмкость,NEUT,0.00,0.167,0.000,0.690,0.143,0.00
46123,ёрзать,NGTV,-0.54,0.050,0.446,0.397,0.107,0.00
46124,ёрничать,NGTV,-0.79,0.078,0.529,0.236,0.157,0.00
46125,ёрш,NEUT,0.16,0.224,0.072,0.576,0.128,0.00


In [33]:
sentiment_vocab["tag"].value_counts()

NEUT    28049
NGTV    11863
PSTV     6215
Name: tag, dtype: int64

In [34]:
with open ('kartaslovsent.csv', encoding = 'utf-8') as csvfile:
    f = csv.reader(csvfile, delimiter=';')
    sent_dict = {}
    for row in f:
        if row[1] == 'NEUT':
            sent_dict[row[0]] = 'neutral'
        elif row[1] == 'NGTV':
            sent_dict[row[0]] = 'negative'
        elif row[1] == 'PSTV':
            sent_dict[row[0]] = 'positive'

Появилась также идея выделить ключевые слова в каждом тексте, что могло бы помочь найти аспекты. Ниже видно, что 3-граммные ключевые слова довольно неплохо выделяют специфичные для отрасли слова и выражения. Их можно использовать для нахождения аспекта Whole и Service (слова более общей тематики, встречаются глаголы, которые могут характеризовать сервис).

In [35]:
from keybert import KeyBERT


In [36]:
kw_model = KeyBERT('clips/mfaq')

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [37]:
train_texts_list_1 = ' '.join(train_texts_list)
kbert_text1_1 = kw_model.extract_keywords(train_texts_list[0], keyphrase_ngram_range=(1, 3), stop_words=None)

In [38]:
kbert_text1_1

[('хороший интерьер', 0.9745),
 ('любить заведение именно', 0.9743),
 ('любить заведение', 0.9735),
 ('который заказать понравиться', 0.9733),
 ('план хороший интерьер', 0.9731)]

In [39]:
kbert_text1_2 = kw_model.extract_keywords(train_texts_list[1], keyphrase_ngram_range=(1, 3), stop_words=None)
kbert_text1_2

[('хороший блюдо кухня', 0.9779),
 ('ресторан согласный многий', 0.9778),
 ('посетить данный ресторан', 0.9774),
 ('хороший блюдо', 0.9771),
 ('мы хороший блюдо', 0.9768)]

### Выделение аспекта и тональности для каждого аспекта

In [51]:
def keybert_extractor(text):
    kbert_text = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words=None)
    key_bert_list = []
    key_bert_list_tokens = []
    for elem in kbert_text:
        key_bert_list.append(elem[0])
        key_bert_list_tokens.extend(elem[0].split(' '))
    return key_bert_list, key_bert_list_tokens


Выделяем аспектные слова и выражения:

In [111]:
def cat_mentions_extractor(name_file):
    texts, ids, norm_texts, norm_tokenized_texts = [], [], [], []
    list_of_final_data = [] #будет хранить список для извлеченных аспектных слов/выражений в формате [id текста, аспект, слово, спан_нач, спан_кон, тональность]
    with open(name_file, encoding = 'utf-8') as f:
        for line in f:
            text_id, text = line.rstrip('\r\n').split('\t')
            texts.append(text) #необработанные отзывы списком
            ids.append(text_id) #id отзывов
            norm_texts.append(normalize_text(text)) #список токенизированных и лемматизированных текстов (каждый отзыв - строка)
            norm_tokenized_texts.append(normalize_text(text).split(' ')) #список токенизированных и лемматизированных текстов (каждый отзыв - список токенов)
    for review_not_split in norm_texts: #берем токенизированные отзывы (1 отзыв - 1 строка)
        key_bert_list, key_bert_list_tokens = keybert_extractor(review_not_split)
        review_check_list = []
        for mention in counted_cat_for_mention.keys():
            if mention in review_not_split:
                for elem in mention.split(' '):
                    if elem in dishes_list and elem in norm_tokenized_texts[norm_texts.index(review_not_split)]:
                        text_id = ids[norm_texts.index(review_not_split)] #получаем id отзыва
                        cat = 'Food' #аспект
                        word = mention #употребление
                        ind = norm_tokenized_texts[norm_texts.index(review_not_split)].index(elem) #индекс токена в предложении, добавляется временно в список с наименованиями аспектов
                        f =  ''.join(norm_tokenized_texts[norm_texts.index(review_not_split)][0:ind])
                        span_beg = len(f) #спан начала
                        span_end = span_beg + len(mention) #спан конца
                        ment_data = [ind, text_id,cat, word, span_beg, span_end]
                        list_of_final_data.append(ment_data)
                        review_check_list.extend(ment_data[3].split(' ')) #добавляем рассмотренное слово в словарь, чтобы не повторяться
                    elif elem in furniture_list and elem in norm_tokenized_texts[norm_texts.index(review_not_split)]:
                        text_id = ids[norm_texts.index(review_not_split)] 
                        cat = 'Interior' 
                        word = mention 
                        ind = norm_tokenized_texts[norm_texts.index(review_not_split)].index(elem)
                        f =  ''.join(norm_tokenized_texts[norm_texts.index(review_not_split)][0:ind])
                        span_beg = len(f) 
                        span_end = span_beg + len(mention)
                        ment_data = [ind, text_id,cat, word, span_beg, span_end]
                        review_check_list.extend(ment_data[3].split(' '))
                        list_of_final_data.append(ment_data)
                    elif elem in norm_tokenized_texts[norm_texts.index(review_not_split)]:
                        text_id = ids[norm_texts.index(review_not_split)] 
                        cat = counted_cat_for_mention[mention] 
                        word = mention 
                        ind = norm_tokenized_texts[norm_texts.index(review_not_split)].index(elem)
                        f =  ''.join(norm_tokenized_texts[norm_texts.index(review_not_split)][0:ind])
                        span_beg = len(f) 
                        span_end = span_beg + len(mention)
                        ment_data = [ind, text_id,cat, word, span_beg, span_end]
                        review_check_list.extend(ment_data[3].split(' '))
                        list_of_final_data.append(ment_data)
        for token in norm_tokenized_texts[norm_texts.index(review_not_split)]:
            if token not in review_check_list:
                if token in dishes_list:
                    text_id = ids[norm_texts.index(review_not_split)] 
                    cat = 'Food' 
                    word = token 
                    ind = norm_tokenized_texts[norm_texts.index(review_not_split)].index(token)
                    f =  ''.join(norm_tokenized_texts[norm_texts.index(review_not_split)][0:ind])
                    span_beg = len(f) 
                    span_end = span_beg + len(token)
                    ment_data = [ind, text_id,cat, word, span_beg, span_end]
                    review_check_list.extend(ment_data[3])
                    list_of_final_data.append(ment_data)
                elif token in furniture_list:
                    text_id = ids[norm_texts.index(review_not_split)] 
                    cat = 'Interior' 
                    word = token 
                    ind = norm_tokenized_texts[norm_texts.index(review_not_split)].index(token)
                    f =  ''.join(norm_tokenized_texts[norm_texts.index(review_not_split)][0:ind])
                    span_beg = len(f) 
                    span_end = span_beg + len(token)
                    ment_data = [ind, text_id,cat, word, span_beg, span_end]
                    review_check_list.extend(ment_data[3])
                    list_of_final_data.append(ment_data)
                elif token in rare_words: #список слов с самым большим tf*idf
                    if token.isdigit:
                        text_id = ids[norm_texts.index(review_not_split)] 
                        cat = 'Price' 
                        word = token 
                        ind = norm_tokenized_texts[norm_texts.index(review_not_split)].index(token)
                        f =  ''.join(norm_tokenized_texts[norm_texts.index(review_not_split)][0:ind])
                        span_beg = len(f) 
                        span_end = span_beg + len(token)
                        ment_data = [ind,text_id,cat, word, span_beg, span_end]
                        review_check_list.extend(ment_data[3])
                        list_of_final_data.append(ment_data)
                    elif 'NOUN' in m.parse(token)[0].tag:
                        text_id = ids[norm_texts.index(review_not_split)] 
                        cat = 'Whole' 
                        word = token 
                        ind = norm_tokenized_texts[norm_texts.index(review_not_split)].index(token)
                        f =  ''.join(norm_tokenized_texts[norm_texts.index(review_not_split)][0:ind])
                        span_beg = len(f) 
                        span_end = span_beg + len(token)
                        ment_data = [ind, text_id,cat, word, span_beg, span_end]
                        review_check_list.extend(ment_data[3])
                        list_of_final_data.append(ment_data)
                elif token in key_bert_list_tokens:
                    if 'NOUN' in m.parse(token)[0].tag:
                        text_id = ids[norm_texts.index(review_not_split)] 
                        cat = 'Whole' 
                        word = token 
                        ind = norm_tokenized_texts[norm_texts.index(review_not_split)].index(token)
                        f =  ''.join(norm_tokenized_texts[norm_texts.index(review_not_split)][0:ind])
                        span_beg = len(f) 
                        span_end = span_beg + len(token)
                        ment_data = [ind, text_id,cat, word, span_beg, span_end]
                        review_check_list.extend(ment_data[3])
                        list_of_final_data.append(ment_data)
                    elif 'VERB' in m.parse(token)[0].tag:
                        text_id = ids[norm_texts.index(review_not_split)] 
                        cat = 'Service' 
                        word = token 
                        ind = norm_tokenized_texts[norm_texts.index(review_not_split)].index(token)
                        f =  ''.join(norm_tokenized_texts[norm_texts.index(review_not_split)][0:ind])
                        span_beg = len(f) 
                        span_end = span_beg + len(token)
                        ment_data = [ind, text_id,cat, word, span_beg, span_end]
                        review_check_list.extend(ment_data[3])
                        list_of_final_data.append(ment_data)
    return texts, ids, norm_texts, norm_tokenized_texts, list_of_final_data
    

In [112]:
texts, ids, norm_texts, norm_tokenized_texts, list_of_final_data = cat_mentions_extractor('dev_reviews.txt')

In [113]:
list_of_final_data[0:20]

[[47, '37784', 'Service', 'обслуживание', 259, 271],
 [5, '37784', 'Whole', 'заведение', 25, 34],
 [18, '37784', 'Food', 'кухня', 97, 102],
 [76, '37784', 'Whole', 'место', 435, 440],
 [25, '37784', 'Food', 'блюдо', 135, 140],
 [62, '37784', 'Service', 'персонал', 357, 365],
 [30, '37784', 'Food', 'порция', 160, 166],
 [13, '37784', 'Food', 'пиво', 74, 78],
 [31, '37784', 'Food', 'салат', 166, 171],
 [24, '37784', 'Food', 'горячий', 128, 135],
 [26, '37784', 'Food', 'мясо', 140, 144],
 [72, '37784', 'Interior', 'стол', 416, 420],
 [65, '37784', 'Service', 'человек', 376, 383],
 [31, '37784', 'Food', 'салат цезарь', 166, 178],
 [32, '37784', 'Food', 'салат цезарь', 171, 183],
 [24, '37784', 'Food', 'горячий блюдо', 128, 141],
 [25, '37784', 'Food', 'горячий блюдо', 135, 148],
 [42, '37784', 'Food', 'попробовать', 224, 235],
 [19, '37784', 'Food', 'есть', 102, 106],
 [32, '37784', 'Food', 'цезарь', 171, 177]]

Для каждого аспектного слова создан словарик в формате: номер слова в предложении (временно, для удобства, потом уберем), id текста, аспект, слово/выражение, спан начала, спан конца.

Затем выделяем для каждого аспекта тональность (к списку для слова добавляем также определение тональности):

In [114]:
def sent_extractor(texts, ids, norm_texts, norm_tokenized_texts, list_of_final_data):
    sent_extracted_data = []
    for line in list_of_final_data:
        #line[0] - индекс слова в предложении отзыва
        #line[1] - id отзыва
        #line[2] - аспектная категория
        #line[3] - слово(сочетание)
        #line[4] - спан начала
        #line[5] - спан конца
        if line[0]>0 and 'ADJF' in m.parse(norm_tokenized_texts[ids.index(line[1])][line[0]-1])[0].tag:
            adj = norm_tokenized_texts[ids.index(line[1])][line[0]-1]
            if adj in sent_dict.keys():
                line.append(sent_dict[adj])
                sent_extracted_data.append(line)
        elif line[0]+1 < len(norm_tokenized_texts[ids.index(line[1])]) and 'NOUN' in m.parse(norm_tokenized_texts[ids.index(line[1])][line[0]+1])[0].tag:
            noun = norm_tokenized_texts[ids.index(line[1])][line[0]+1]
            if noun in sent_dict.keys():
                line.append(sent_dict[noun])
                sent_extracted_data.append(line)
        elif line[0]+1 < len(norm_tokenized_texts[ids.index(line[1])]) and 'VERB' in m.parse(norm_tokenized_texts[ids.index(line[1])][line[0]+1])[0].tag:
            verb = norm_tokenized_texts[ids.index(line[1])][line[0]+1]
            if verb in sent_dict.keys():
                line.append(sent_dict[verb])
                sent_extracted_data.append(line)
        elif line[3] in counted_sent_for_mention.keys():
            line.append(counted_sent_for_mention[line[3]])
            sent_extracted_data.append(line)
        elif line[3] in sent_dict.keys():
                line.append(sent_dict[line[3]])
                sent_extracted_data.append(line)
        else:
            line.append('neutral')
            sent_extracted_data.append(line)
    return sent_extracted_data

Получившийся список:

In [115]:
sent_extracted_data = sent_extractor(texts, ids, norm_texts, norm_tokenized_texts, list_of_final_data)
sent_extracted_data

[[47, '37784', 'Service', 'обслуживание', 259, 271, 'positive'],
 [18, '37784', 'Food', 'кухня', 97, 102, 'positive'],
 [76, '37784', 'Whole', 'место', 435, 440, 'positive'],
 [25, '37784', 'Food', 'блюдо', 135, 140, 'neutral'],
 [62, '37784', 'Service', 'персонал', 357, 365, 'positive'],
 [30, '37784', 'Food', 'порция', 160, 166, 'neutral'],
 [31, '37784', 'Food', 'салат', 166, 171, 'neutral'],
 [24, '37784', 'Food', 'горячий', 128, 135, 'neutral'],
 [26, '37784', 'Food', 'мясо', 140, 144, 'positive'],
 [72, '37784', 'Interior', 'стол', 416, 420, 'positive'],
 [65, '37784', 'Service', 'человек', 376, 383, 'positive'],
 [31, '37784', 'Food', 'салат цезарь', 166, 178, 'neutral'],
 [32, '37784', 'Food', 'салат цезарь', 171, 183, 'positive'],
 [24, '37784', 'Food', 'горячий блюдо', 128, 141, 'neutral'],
 [25, '37784', 'Food', 'горячий блюдо', 135, 148, 'neutral'],
 [42, '37784', 'Food', 'попробовать', 224, 235, 'positive'],
 [32, '37784', 'Food', 'цезарь', 171, 177, 'both'],
 [9, '37784',

In [135]:
sent_extracted_data_final = []
for line in sent_extracted_data:
    sent_extracted_data_final.append(line[1:7])
sent_extracted_data_final

[['37784', 'Service', 'обслуживание', 259, 271, 'positive'],
 ['37784', 'Food', 'кухня', 97, 102, 'positive'],
 ['37784', 'Whole', 'место', 435, 440, 'positive'],
 ['37784', 'Food', 'блюдо', 135, 140, 'neutral'],
 ['37784', 'Service', 'персонал', 357, 365, 'positive'],
 ['37784', 'Food', 'порция', 160, 166, 'neutral'],
 ['37784', 'Food', 'салат', 166, 171, 'neutral'],
 ['37784', 'Food', 'горячий', 128, 135, 'neutral'],
 ['37784', 'Food', 'мясо', 140, 144, 'positive'],
 ['37784', 'Interior', 'стол', 416, 420, 'positive'],
 ['37784', 'Service', 'человек', 376, 383, 'positive'],
 ['37784', 'Food', 'салат цезарь', 166, 178, 'neutral'],
 ['37784', 'Food', 'салат цезарь', 171, 183, 'positive'],
 ['37784', 'Food', 'горячий блюдо', 128, 141, 'neutral'],
 ['37784', 'Food', 'горячий блюдо', 135, 148, 'neutral'],
 ['37784', 'Food', 'попробовать', 224, 235, 'positive'],
 ['37784', 'Food', 'цезарь', 171, 177, 'both'],
 ['37784', 'Whole', 'вечером', 46, 53, 'positive'],
 ['37784', 'Food', 'выбор', 9

In [130]:
def review_sent_estimate(sent_extracted_data):
    

### Шаблон для тестирования программы

In [ ]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [ ]:
punct = '/.,:;!?"<>(){}[]*-+'
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        if t not in punct:
            lemmas.append(
                m.parse(t)[0].normal_form
            )
    return ' '.join(lemmas)

In [ ]:
with open('tfidf_data.txt', 'w') as f:
    rare_words = f.read().split(' ')

In [ ]:
with open('furnit_list.txt', encoding = 'utf-8') as file:
    f = file.readlines()
    furniture_list = []
    for line in f:
        line = line.rstrip('\r\n').lower()
        furniture_list.append(line)

In [ ]:
with open('dishes_list_from_wiki.txt', encoding = 'utf-8') as file2:
    f2 = file2.readlines()
    dishes_list = []
    for line in f2:
        line = line.rstrip('\r\n').lower()
        dishes_list.append(line)

In [ ]:
with open ('kartaslovsent.csv', encoding = 'utf-8') as csvfile:
    f = csv.reader(csvfile, delimiter=';')
    sent_dict = {}
    for row in f:
        if row[1] == 'NEUT':
            sent_dict[row[0]] = 'neutral'
        elif row[1] == 'NGTV':
            sent_dict[row[0]] = 'negative'
        elif row[1] == 'PSTV':
            sent_dict[row[0]] = 'positive'

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT('clips/mfaq')

In [ ]:
def keybert_extractor(text):
    kbert_text = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words=None)
    key_bert_list = []
    key_bert_list_tokens = []
    for elem in kbert_text:
        key_bert_list.append(elem[0])
        key_bert_list_tokens.extend(elem.split(' '))
    return key_bert_list, key_bert_list_tokens


In [ ]:
texts, ids, norm_texts, norm_tokenized_texts, list_of_final_data = cat_mentions_extractor(name_of_file)

In [ ]:
sent_extracted_data = sent_extractor (list_of_final_data)
sent_extracted_data_final = []
for line in sent_extracted_data:
    sent_extracted_data_final.append(line[1:7])
sent_extracted_data_final

In [ ]:
review_sent_estimate